
# Slipcase Price-Function Tuner  
*   Start by running all notebook cells using the toolbar above: <b>Runtime --> Run all</b>, or run each individual cell in order by clicking the **play** button in the upper-left of the cell when you hover over it.</b>
*   The linear model is the base: <i>$(per\_linear\_inch)*(length+width+height)$</i>
*   In the <b>MODIFY THESE VALUES ONLY</b> area below, set <i>per_linear_inch</i> to .5 or wherever you want to start, then run that individual cell by clicking the **play** button.
*   You'll see the price table generated below and also a table that shows the impact of any adjustments (at first, this should be a grid of all 0, if you haven't made any adjustments).
*   Below the price table is the mathmatical expression of the price function in PHP code. You can copy and paste to use where you need it, e.g. &nbsp; &nbsp;  <i> $ceil((0.5)*(\$length + \$height + \$width)$</i>)
*   The current values for all the variables are also listed, so you can note them to recreate the table in another session.
*   Try modifying the <b>CORNER ADJUSTMENTS</b> values with positive or negative values to raise or lower prices in specific parts of the table, then click the **play** button again to run the cell and update the price table, impact table and the mathematical expression. Start with small, incremental changes of +/- .01
* Try different combinations of corner adjustments and different values for <i>per_linear_inch</i> to tune the prices.
*  You can also "blend in" a square-inch base-model <i>$(per\_sq\_inch)*((length+2width)*(height+2width))$</i> by changing the value of the <i>per_sq_inch</i> variable.   





In [ ]:
# @title
from IPython.display import HTML
import pandas as pd
import math

def get_linear_inch_price (length, height, **kwargs):

  return math.ceil((length + height + kwargs["width"]) * kwargs["per_linear_inch"])  #tuned with .46

def get_sq_inch_price (length, height, **kwargs):

  return math.ceil(
      (
      (length*height)
      +(2*length*kwargs["width"])
      +(2*height*kwargs["width"])
      +(4*((kwargs["width"])**2))
      )
      *kwargs["per_sq_inch"]
      )

def get_blend_price(length, height, **kwargs):

    return math.ceil( get_sq_inch_price(length, height, **kwargs)
                    + get_linear_inch_price(length, height, **kwargs)
                    )

def get_blend_price2(length, height, **kwargs):

    return math.ceil( get_sq_inch_price(length, height, **kwargs)
                    + get_linear_inch_price(length, height, **kwargs)
                    + 1.06**(95-(length+height))*kwargs["top_left"] #moves top left quadrant, tuned with 95
                    + 1.06**((length+height)-140)*kwargs["bottom_right"]  #moves bottom right quadrant, tuned with 140
                    + 1.06**((length-height)-40)*kwargs["top_right"]  #moves top right quadrant, tuned with 40
                    + 1.06**((height-length)-10)*kwargs["bottom_left"]  #moves bottom left quadrant, tuned with 10
                    )

def gen_table(model, **kwargs):
    cols=[x for x in range(12,121,6)]
    index=[x for x in range(12,91,6)]
    prices=[]

    for h in range(12,91,6):
      row=[]
      for l in range(12,121,6):
        row.append(model(l,h,**kwargs))
      prices.append(row)

    return prices, cols, index

def price_model_str(**kwargs):
    top_left_str = ""
    bottom_right_str = ""
    top_right_str = ""
    bottom_left_str = ""

    if kwargs["top_left"]!=0:
      top_left_str = " + " + "(" + str(kwargs["top_left"]) + ")" + "*( 1.06^(95-(length + height)) )"

    if kwargs["bottom_right"]!=0:
      bottom_right_str = " + " + "(" + str(kwargs["bottom_right"]) + ")" + "*( 1.06^((length + height)-140) )"

    if kwargs["top_right"]!=0:
      top_right_str = " + " + "(" + str(kwargs["top_right"]) + ")" + "*( 1.06^((length - height)-40) )"

    if kwargs["bottom_left"]!=0:
      bottom_left_str = " + " + "(" + str(kwargs["bottom_left"]) + ")" + "*( 1.06^((height - length)-10) )"

    corners_str = top_left_str + bottom_right_str + top_right_str + bottom_left_str


    if kwargs["per_sq_inch"]==0:  #show linear_inch model
      model = (
          "ceiling("
              + "\n  (" + str(kwargs["per_linear_inch"]) + ")" + "*(length + height + width)"
              + corners_str
              + "\n)"
            )
    elif kwargs["per_linear_inch"]==0:  #show sq_inch model
        model = (
          "ceiling("
              + "\n  (" + str(kwargs["per_sq_inch"]) + ")" + "*( (length*height) + (2*width)*(length + height) + (4)*(width^2) )"
              + corners_str
              + "\n)"
            )
    else:     #show linear_inch / sq_inch models blend
        model = (
          "ceiling("
              + "\n  (" + str(kwargs["per_linear_inch"]) + ")" + "*(length + height + width)"
              + "\n   + (" + str(kwargs["per_sq_inch"]) + ")" + "*( (length*height) + (2*width)*(length + height) + (4)*(width^2) )"
              +  "\n    " + corners_str
              + "\n)"
            )

    return model

def price_model_str_php(**kwargs):
    top_left_str = ""
    bottom_right_str = ""
    top_right_str = ""
    bottom_left_str = ""

    if kwargs["top_left"]!=0:
      top_left_str = " + " + "(" + str(kwargs["top_left"]) + ")" + "*(1.06**(95-($length + $height)))"

    if kwargs["bottom_right"]!=0:
      bottom_right_str = " + " + "(" + str(kwargs["bottom_right"]) + ")" + "*(1.06**(($length + $height)-140))"

    if kwargs["top_right"]!=0:
      top_right_str = " + " + "(" + str(kwargs["top_right"]) + ")" + "*(1.06**(($length - $height)-40))"

    if kwargs["bottom_left"]!=0:
      bottom_left_str = " + " + "(" + str(kwargs["bottom_left"]) + ")" + "*(1.06**(($height - $length)-10))"

    corners_str = top_left_str + bottom_right_str + top_right_str + bottom_left_str


    if kwargs["per_sq_inch"]==0:  #show linear_inch model
      model = (
          "ceil("
              + "(" + str(kwargs["per_linear_inch"]) + ")" + "*($length + $height + $width)"
              + corners_str
              + ")"
            )
    elif kwargs["per_linear_inch"]==0:  #show sq_inch model
        model = (
          "ceil("
              + "(" + str(kwargs["per_sq_inch"]) + ")" + "*(($length * $height) + (2 * $width)*($length + $height) + (4*($width**2)))"
              + corners_str
              + ")"
            )
    else:     #show linear_inch / sq_inch models blend
        model = (
          "ceil("
              + "(" + str(kwargs["per_linear_inch"]) + ")" + "*($length + $height + $width)"
              + " + (" + str(kwargs["per_sq_inch"]) + ")" + "*( ($length * $height) + (2 * $width)*($length + $height) + (4*($width**2)))"
              + corners_str
              + ")"
            )

    return model

def run_table(**values):
  if values["tuning_table"] == 0:
    print_final_table(**values)
  elif values["tuning_table"] == 1:
    print_movement_table(**values)
  else:
    print_all_tables(**values)


def print_all_tables(**values):

    table_values=gen_table(get_sq_inch_price, **values)
    df1 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    print("\n"+" PURE SQUARE INCH PRICES:")
    display(df1)

    print("\n"+" PURE LINEAR PRICES:")

    table_values=gen_table(get_linear_inch_price, **values)
    df2 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    display(df2)

    table_values=gen_table(get_blend_price, **values)
    df3 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    print("\n"+" BLEND of SQ INCH and LINEAR PRICES without CORNER ADJUSTMENTS:")
    display(df3)

    table_values=gen_table(get_blend_price2, **values)
    df4 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    #print("\n"+" MOVEMENT caused by the CORNER ADJUSTMENTS:")
    #display(df4-df3)

    print("\n"+" MOVEMENT caused by the CORNER ADJUSTMENTS and SQ INCH BLEND:")
    display( (df4-df3)+ df1)

    print("\n\n"+" **TUNED FINAL PRICES** BLEND of SQ INCH and LINEAR PRICES with CORNER ADJUSTMENTS:")
    display(df4)

    print("\n" + "TUNED FINAL PRICE MODEL PHP CODE:\n\n" + price_model_str_php(**values))

def print_final_table(**values):

    table_values=gen_table(get_blend_price2, **values)
    df4 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    print("\n\n"+" **TUNED FINAL PRICES** BLEND of SQ INCH and LINEAR PRICES with CORNER ADJUSTMENTS:")
    display(df4)

    print("\n" + "TUNED FINAL PRICE MODEL PHP CODE:\n\n" + price_model_str_php(**values))

def print_movement_table(**values):

    table_values=gen_table(get_sq_inch_price, **values)
    df1 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    table_values=gen_table(get_linear_inch_price, **values)
    df2 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    table_values=gen_table(get_blend_price, **values)
    df3 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

    table_values=gen_table(get_blend_price2, **values)
    df4 = pd.DataFrame(table_values[0], columns=table_values[1], index = table_values[2])

   # print("\n"+" MOVEMENT caused by the CORNER ADJUSTMENTS and SQ INCH BLEND:")
   # display( (df4-df3)+ df1)

   # print("\n\n"+" **TUNED FINAL PRICES** BLEND of SQ INCH and LINEAR PRICES with CORNER ADJUSTMENTS:")
   # display(df4)

    print("**TUNED FINAL PRICES** BLEND of SQ INCH and LINEAR PRICES with CORNER ADJUSTMENTS                     MOVEMENT caused by the CORNER ADJUSTMENTS and SQ INCH BLEND")

    side_by_side(df4,((df4-df3)+df1))

    print("\n" + "TUNED FINAL PRICE MODEL PHP CODE:\n\n" + price_model_str_php(**values))
    print("\n\n" + str(values))

def side_by_side(*dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 2em">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

def stored_model(num):
  if num == 1:
    values={'width': 2, 'per_sq_inch': 0.009, 'per_linear_inch': -0.31, 'top_left': 2.5, 'top_right': 10, 'bottom_left': 1.7999999999999998, 'bottom_right': 2}
    print("CENTER HOLE MODEL")
    print_final_table(**values)
    print("\n" + str(values))


In [ ]:
#~~~~~~~~MODIFY THESE VALUES ONLY, then run cell~~~~~~~~~~~~~
#SLIPCASE WIDTH
width = 2                                 #width of the slipcase; range: >0-8

#BASE MODELS
per_linear_inch = 0.55                   #the primary linear-inch model: (per_linear_inch)*(L+W+H)
per_sq_inch = 0.00                        #start with 0; a supplemental square-inch model to blend in; price increases rapidly as area increases; .0013 will add $1 to 12x12 and $16 to 90X120


#CORNER ADJUSTMENTS                      #try increments of +/- .01 for small adjustments
top_left = 0.00                           #start with 0;
top_right_and_bottom_left = 0.00         #start with 0; the bottom left will move at a fixed ratio of.18 of top right value
bottom_right = 0.00                     #start with 0;


#OUTPUT MODE
print_tuning_tables = 1                   #set to 0 to only show the final price table; 1 to show side-by-side final table and adjusment impact, 2 to show all tuning tables
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

values={"width": width, "per_sq_inch": per_sq_inch, "per_linear_inch": per_linear_inch, "top_left": top_left, "top_right": top_right_and_bottom_left, "bottom_left": top_right_and_bottom_left*.18, "bottom_right": bottom_right, "tuning_table": print_tuning_tables}
run_table(**values)

**TUNED FINAL PRICES** BLEND of SQ INCH and LINEAR PRICES with CORNER ADJUSTMENTS                     MOVEMENT caused by the CORNER ADJUSTMENTS and SQ INCH BLEND


,12,18,24,30,36,42,48,54,60,66,72,78,84,90,96,102,108,114,120
12,15,18,21,25,28,31,35,38,41,44,48,51,54,58,61,64,68,71,74
18,18,21,25,28,31,35,38,41,44,48,51,54,58,61,64,68,71,74,77
24,21,25,28,31,35,38,41,44,48,51,54,58,61,64,68,71,74,77,81
30,25,28,31,35,38,41,44,48,51,54,58,61,64,68,71,74,77,81,84
36,28,31,35,38,41,44,48,51,54,58,61,64,68,71,74,77,81,84,87
42,31,35,38,41,44,48,51,54,58,61,64,68,71,74,77,81,84,87,91
48,35,38,41,44,48,51,54,58,61,64,68,71,74,77,81,84,87,91,94
54,38,41,44,48,51,54,58,61,64,68,71,74,77,81,84,87,91,94,97
60,41,44,48,51,54,58,61,64,68,71,74,77,81,84,87,91,94,97,101
66,44,48,51,54,58,61,64,68,71,74,77,81,84,87,91,94,97,101,104



TUNED FINAL PRICE MODEL PHP CODE:

ceil((0.55)*($length + $height + $width))


{'width': 2, 'per_sq_inch': 0.0, 'per_linear_inch': 0.55, 'top_left': 0.0, 'top_right': 0.0, 'bottom_left': 0.0, 'bottom_right': 0.0, 'tuning_table': 1}


In [ ]:
#print an individual price using the tuned model and width from the cell above
length = 0.00
height = 0.00

print("$" + str(get_blend_price2(length, height, **values)))

$2


In [ ]:
#show a stored model
#CENTER HOLE MODEL = 1   just for fun to demonstrate how far the prices can be modified
stored_model(0)